In [1]:
import pandas as pd
df = pd.read_csv("Doc\\ipodata.csv")
df.head()

,Code,Name,Date,Price,Share,Amount,Warran,Underwriter,Sector,Subsector,Field,Website,Prospectus,Summary_Prospectus,Alamat,Image,Papan Pencatatan
0,GUNA,PT. Gunanusa Eramandiri Tbk.,09 Jul 2024,Rp150,20%,"5,000,000 lot",0,GR - PANIN SEKURITAS TBK.,Barang Konsumen Primer,Makanan & Minuman,Industri makanan dari kedele dan kacang-kacang...,https://www.gunanusaeramandiri.com,https://storage.googleapis.com/assets.trading....,https://storage.googleapis.com/assets.trading....,Kawasan Industri BIIE Hyundai Jl. Inti I Kav. ...,https://storage.googleapis.com/assets.trading....,Pengembangan
1,GOLF,PT Intra GolfLink Resorts Tbk,08 Jul 2024,Rp200,15.02%,"31,000,000 lot",0,IF - SAMUEL SEKURITAS INDONESIA,Barang Konsumen Non-Primer,Barang Rekreasi,Bergerak Dalam Bidang Jasa Konsultasi Manajeme...,https://www.golflinkresorts.co.id/id,https://storage.googleapis.com/assets.trading....,https://storage.googleapis.com/assets.trading....,"Desa Kadumanggu, Kecamatan Madang Bogor 16810 ...",https://storage.googleapis.com/assets.trading....,Utama
2,LABS,PT UBC Medical Indonesia Tbk,10 Jul 2024,Rp102,17.72%,"7,000,000 lot",0,YJ - LOTUS ANDALAN SEKURITAS,Kesehatan,Jasa & Peralatan Kesehatan,Distributor Alat Kesehatan,https://www.ubcindonesia.com,https://storage.googleapis.com/assets.trading....,https://storage.googleapis.com/assets.trading....,"Kawasan Industri Pulogadung, Jl. Rawagelam V, ...",https://storage.googleapis.com/assets.trading....,Pengembangan
3,ISEA,PT Indo American Seafoods Tbk,08 Jul 2024,Rp250,20.86%,"2,900,000 lot",2 : 1,CP - VALBURY SEKURITAS INDONESIA,Barang Konsumen Primer,Makanan & Minuman,"Industri Pembekuan Biota Air Lainnya, Industri...",https://www.indoamericanseafoods.com/,https://storage.googleapis.com/assets.trading....,https://storage.googleapis.com/assets.trading....,"Jl. Ir. Sutami KM 13, Desa Sukanegara, Kecamat...",https://storage.googleapis.com/assets.trading....,Pengembangan
4,PART,PT Cipta Perdana Lancar,05 Jul 2024,Rp105,25%,"6,800,000 lot",1 : 1,EP - MNC SEKURITAS,Barang Konsumen Non-Primer,Otomotif & Komponen Otomotif,Perseroan bergerak di bidang Manufaktur Kompon...,https://www.ciptaperdanalancar.com,https://storage.googleapis.com/assets.trading....,https://storage.googleapis.com/assets.trading....,"Jl. Prabu Siliwangi KM 0,5 Keroncong, Kec. Jat...",https://storage.googleapis.com/assets.trading....,Pengembangan


In [2]:
df["Price"] = df["Price"].str.replace("Rp","").astype(int)
prices = df["Price"]
df["Share"] = df["Share"].str.replace("%","").astype(float)
df["Warran"] = df['Warran'].apply(lambda x: int(x.split(":")[0])/int(x.split(":")[1]) if x != "0" else int(x) )
df["Underwriter"] = df["Underwriter"].apply(lambda x:x.split("-")[0])
df["Amount"] = df["Amount"].str.replace(",","")
df["Amount"] = df["Amount"].str.replace("lot","")
df["Amount"] = df["Amount"].apply(lambda x:int(x))

In [3]:
from sklearn.preprocessing import LabelEncoder
# Initialize the LabelEncoder
encoder = LabelEncoder()
# Encode each column
for column in ["Papan Pencatatan","Underwriter","Sector","Subsector"]:
    df[column] = encoder.fit_transform(df[column])

In [4]:
feature_enginering = ["Papan Pencatatan","Share","Underwriter","Warran","Sector","Subsector","Price","Amount"]

In [5]:
x = df[feature_enginering]

In [6]:
target = pd.read_csv("Doc//ipo_gain.csv")
target.head()

,1 candle gain,2 candle gain,3 candle gain,4 candle gain,5 candle gain,1 candle high,2 candle high,3 candle high,4 candle high,5 candle high
0,34.666667,68.000000,109.333333,98.666667,94.666667,34.666667,68.000000,109.333333,161.333333,116.000000
1,35.000000,36.000000,20.000000,20.000000,15.000000,35.000000,51.000000,38.000000,31.000000,21.000000
2,34.313725,80.392157,92.156863,66.666667,76.470588,34.313725,80.392157,123.529412,109.803922,88.235294
3,24.800000,28.000000,52.000000,87.200000,40.800000,24.800000,40.000000,52.000000,88.000000,98.400000
4,24.761905,4.761905,10.476190,16.190476,22.857143,34.285714,28.571429,19.047619,18.095238,22.857143


In [7]:
def to_categoric(x):
    j=0
    if x < 1:
        return 0
    for i in range(1,280,5):
        j+=1
        if i <= x < i+5:
            print(f"{i} <= {x:.2f} < {i+5} === {j}" )
            return j

        
for label in target.columns:
    target[label] = target[label].apply(lambda x:to_categoric(x))

31 <= 34.67 < 36 === 7
31 <= 35.00 < 36 === 7
31 <= 34.31 < 36 === 7
21 <= 24.80 < 26 === 5
21 <= 24.76 < 26 === 5
31 <= 34.43 < 36 === 7
6 <= 9.60 < 11 === 2
6 <= 9.09 < 11 === 2
31 <= 34.55 < 36 === 7
31 <= 34.04 < 36 === 7
31 <= 35.00 < 36 === 7
31 <= 35.00 < 36 === 7
6 <= 6.11 < 11 === 2
26 <= 27.50 < 31 === 6
16 <= 17.57 < 21 === 4
11 <= 13.79 < 16 === 3
31 <= 34.78 < 36 === 7
6 <= 9.71 < 11 === 2
21 <= 25.00 < 26 === 5
21 <= 24.63 < 26 === 5
1 <= 4.00 < 6 === 1
31 <= 34.29 < 36 === 7
11 <= 14.56 < 16 === 3
11 <= 15.33 < 16 === 3
16 <= 19.86 < 21 === 4
31 <= 34.26 < 36 === 7
31 <= 34.86 < 36 === 7
21 <= 24.85 < 26 === 5
31 <= 35.00 < 36 === 7
6 <= 7.01 < 11 === 2
6 <= 9.70 < 11 === 2
1 <= 4.17 < 6 === 1
6 <= 10.00 < 11 === 2
31 <= 32.81 < 36 === 7
21 <= 25.00 < 26 === 5
31 <= 35.00 < 36 === 7
31 <= 35.00 < 36 === 7
6 <= 9.02 < 11 === 2
11 <= 13.91 < 16 === 3
6 <= 10.00 < 11 === 2
11 <= 13.00 < 16 === 3
31 <= 35.00 < 36 === 7
1 <= 2.00 < 6 === 1
1 <= 3.08 < 6 === 1
31 <= 34.26 < 36

In [8]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import joblib

In [9]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning

# Suppress specific warnings
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)

In [24]:
def predict(label):
    y = target[label]
    X = df[feature_enginering]
    attemp = label

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the models and their hyperparameters for tuning
    models = {
        'LogisticRegression': (LogisticRegression(max_iter=1000), {'C': [0.1, 1, 10]}),
        'DecisionTree': (DecisionTreeClassifier(), {'max_depth': [3, 5, 10]}),
        'RandomForest': (RandomForestClassifier(), {'n_estimators': [50, 100], 'max_depth': [3, 5, 10]}),
        'SVC': (SVC(), {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto']}),
        'XGBClassifier': (XGBClassifier(), {'learning_rate': [0.01, 0.1], 'max_depth': [3, 5], 'n_estimators': [50, 100]})
        }

    # Initialize variables to store the best model
    best_f1 = 0
    best_model = None
    best_model_name = ""
    results_list = []

    # Use the standard scorers in GridSearchCV
    skf = StratifiedKFold(n_splits=5)

    # Use the stratified k-folds in GridSearchCV
    for model_name, (model_class, params) in models.items():
        model = model_class.__class__()
        grid_search = GridSearchCV(model, params, cv=skf, scoring='f1_macro')
        grid_search.fit(X_train, y_train)
        
        best_model_instance = grid_search.best_estimator_
        y_pred = best_model_instance.predict(X_test)
            
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro')
        recall = recall_score(y_test, y_pred, average='macro')
        f1 = f1_score(y_test, y_pred, average='macro')

        results_list.append({
                'Model': model_name,
                'BestParams': grid_search.best_params_,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1 Score': f1
            })

            # Save the best model based on F1 Score
        if f1 > best_f1:
                best_f1 = f1
                best_model = best_model_instance
                best_model_name = model_name

        # Convert results list to a DataFrame
    results = pd.DataFrame(results_list)
        # Save all model results to a CSV file
    results.to_csv(f'Doc//{attemp}_model_results.csv', index=False)
        # Save the best model to a file
    joblib.dump(best_model, f'Doc//{attemp}_best_model.pkl')
    print(f"All model results saved to 'Doc//{attemp}_model_results.csv'.")
    print(f"Best model ({best_model_name}) saved to 'Doc//{attemp}_best_model.pkl' with F1 Score: {best_f1}")

    # Load the best model from the file
    best_model = joblib.load(f'Doc//{attemp}_best_model.pkl')
    X = df[feature_enginering]
    y = y
        # Make predictions using the best model
    y_pred = best_model.predict(X)
        
        # Calculate and print overall metrics
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred, average='macro')
    recall = recall_score(y, y_pred, average='macro')
    f1 = f1_score(y, y_pred, average='macro')
    print(f"Overall Accuracy: {accuracy}")
    print(f"Overall Precision: {precision}")
    print(f"Overall Recall: {recall}")
    print(f"Overall F1 Score: {f1}")
        
        # Generate a classification report
    print(classification_report(y, y_pred))
        
        # Generate a confusion matrix
    conf_matrix = confusion_matrix(y, y_pred)
    print(conf_matrix)
        
        # Save errors to CSV
    errors = pd.DataFrame({
            'Actual': y,
            'Predicted': y_pred
        })

    errors.insert(0, 'Code', df['Code'])
    errors.to_csv(f"Doc//{attemp}_error.csv", index=False)
    print("\n")

In [60]:
target["2 candle high"].value_counts()

2 candle high
0     34
17    23
11    15
14    14
2     13
5     13
3     12
16    11
4     10
1     10
6      7
9      3
8      3
13     3
15     3
10     2
7      2
12     1
Name: count, dtype: int64

In [57]:
vals = {}
def encode_labels(y):
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    encoding_map = dict(zip(le.classes_, le.transform(le.classes_)))
    return y_encoded, encoding_map

for y in target.columns:
    if "high" in y and "1" not in y:
        print(f"{y} before encode {target[y].value_counts()}")
        target[y], vals[y] = encode_labels(target[y])
        print(f"{y} after encode {target[y].value_counts()}")


2 candle high before encode 2 candle high
0     34
17    23
11    15
14    14
2     13
5     13
3     12
16    11
4     10
1     10
6      7
9      3
8      3
13     3
15     3
10     2
7      2
12     1
Name: count, dtype: int64
2 candle high after encode 2 candle high
0     34
17    23
11    15
14    14
2     13
5     13
3     12
16    11
4     10
1     10
6      7
9      3
8      3
13     3
15     3
10     2
7      2
12     1
Name: count, dtype: int64
3 candle high before encode 3 candle high
0     45
1     17
29    14
4     12
5     12
2     10
3     10
7      6
22     5
8      5
19     5
6      4
12     4
13     4
14     3
15     3
25     3
26     3
16     3
23     2
11     2
10     2
9      2
20     1
17     1
18     1
Name: count, dtype: int64
3 candle high after encode 3 candle high
0     45
1     17
25    14
4     12
5     12
2     10
3     10
7      6
21     5
8      5
19     5
6      4
12     4
13     4
14     3
15     3
23     3
24     3
16     3
22     2
11     2
10     2


In [48]:
print(vals)

{'2 candle high': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17}, '3 candle high': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 22: 21, 23: 22, 25: 23, 26: 24, 29: 25}, '4 candle high': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 17: 15, 18: 16, 19: 17, 20: 18, 21: 19, 22: 20, 26: 21, 27: 22, 28: 23, 29: 24, 32: 25, 33: 26, 34: 27, 37: 28, 38: 29, 40: 30, 41: 31}, '5 candle high': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 16: 14, 18: 15, 20: 16, 23: 17, 24: 18, 26: 19, 27: 20, 28: 21, 29: 22, 30: 23, 31: 24, 32: 25, 33: 26, 34: 27, 36: 28, 40: 29, 41: 30, 43: 31, 44: 32, 46: 33, 52: 34, 54: 35, 56: 36}}


In [50]:
target["2 candle high"].value_counts()

2 candle high
0     34
17    23
11    15
14    14
2     13
5     13
3     12
16    11
4     10
1     10
6      7
9      3
8      3
13     3
15     3
10     2
7      2
12     1
Name: count, dtype: int64

In [51]:
for y in target.columns:
    if "high" in y:
        # print(target[y].value_counts())
        predict(y)

All model results saved to 'Doc//1 candle high_model_results.csv'.
Best model (RandomForest) saved to 'Doc//1 candle high_best_model.pkl' with F1 Score: 0.3937074829931973
Overall Accuracy: 0.8324022346368715
Overall Precision: 0.7519417475728156
Overall Recall: 0.586796161014911
Overall F1 Score: 0.6389472732783752
              precision    recall  f1-score   support

           0       0.80      0.73      0.76        11
           1       1.00      0.57      0.73         7
           2       0.90      0.70      0.79        27
           3       0.67      0.29      0.40         7
           4       1.00      0.50      0.67         6
           5       0.83      0.91      0.87        32
           6       0.00      0.00      0.00         5
           7       0.82      1.00      0.90        84

    accuracy                           0.83       179
   macro avg       0.75      0.59      0.64       179
weighted avg       0.82      0.83      0.81       179

[[ 8  0  0  0  0  0  0  3]
 [ 0

ValueError: 
All the 40 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
32 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\xgboost\sklearn.py", line 1491, in fit
    raise ValueError(
ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16], got [ 0  1  2  3  4  5  6  7  8  9